# Morphing

> This module apply morphing to a template source from landmarks. Morphing from pygem library.


In [ ]:
#| default_exp morph

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import numpy as np
import click
from pygem import RBF

In [ ]:
# #| export
from infepy.preprocessing import read_landmarks, read_nodes, _check_landmarks
from infepy.utils import read_toml, multiple_targets, _merge_path

In [ ]:
#| export
def morphing(source_template: np.ndarray, # coordinates of the template mesh 
             source_landmarks: np.ndarray,  # Landmarks of the source template
             target_landmarks: np.ndarray,  # Target Landmarks
             ):
    "Morph the target geometry with RBF function - Thin Plate Spine. Plot the mesh and morphed mesh."
    rbf = RBF(original_control_points=source_landmarks, deformed_control_points=target_landmarks,
            func='thin_plate_spline', radius=1.0)
    return rbf#rbf(source_template)

In [ ]:
#| export
def write_output(mesh: np.ndarray, # Morphed geometry
    ):
    "Write an output file for the morphed geometry in key file format."
    pass
    return

In [ ]:
#| export
def main():
    print(os.getcwd())
    config = read_toml()
    source_landmarks = read_landmarks(config['source']['filename_landmarks'])
    template_geometry = read_nodes(config['source']['filename_geometry'])

    if multiple_targets() == bool: # single target
        target_landmarks = read_landmarks(config['target']['filename_landmarks'])
    # _check_landmarks(source_landmarks, target_landmarks)
        morphed_geometry = morphing(source_landmarks, target_landmarks, template_geometry)
        write_output(morphed_geometry)
    else:
        targets_folder = multiple_targets()
        for folder in targets_folder:
            new_path = os.path.join(config['target']['path'],folder, config['target']['filename_landmarks'])
            target_landmarks = read_landmarks(new_path)
        # _check_landmarks(source_landmarks, target_landmarks)
            morphed_geometry = morphing(source_landmarks, target_landmarks, template_geometry)
            write_output(morphed_geometry)
            print("Yes!!")
    return

In [ ]:
main()

/mnt/c/Users/chiarar/OneDrive - Chalmers/SSMT - Subject Specific Modeling Toolstack/Github/infepy/nbs
Yes!!
Yes!!


/mnt/c/Users/chiarar/OneDrive - Chalmers/SSMT - Subject Specific Modeling Toolstack/Github/infepy/nbs/pygem/rbf_factory.py:93: RuntimeWarning: divide by zero encountered in log
  result = np.where(arg > 0, result * np.log(arg), result)
/mnt/c/Users/chiarar/OneDrive - Chalmers/SSMT - Subject Specific Modeling Toolstack/Github/infepy/nbs/pygem/rbf_factory.py:93: RuntimeWarning: invalid value encountered in multiply
  result = np.where(arg > 0, result * np.log(arg), result)


In [ ]:
#| export
# @click.command()
# def hello():
#     click.echo("hello")

In [ ]:
# #| export
# if __name__ == "__main__":
#     hello()
    

Usage: ipykernel_launcher.py [OPTIONS]
Try 'ipykernel_launcher.py --help' for help.

Error: No such option: --ip Did you mean --help?


AssertionError: 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()